<a href="https://colab.research.google.com/github/Bindhya-K/NLP/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [ ]:
!kaggle datasets download -d algord/fake-news



  0% 0.00/1.68M [00:00<?, ?B/s]
100% 1.68M/1.68M [00:00<00:00, 82.3MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  fake-news.zip
  inflating: FakeNewsNet.csv         


In [3]:
import pandas as pd


In [5]:
df=pd.read_csv('/content/drive/MyDrive/Kaggle/FakeNewsNet.csv')

In [6]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [7]:
df.shape

(23196, 5)

In [8]:
df.isna().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [9]:
df=df.dropna()

In [10]:
df.shape

(22866, 5)

In [11]:
X=df.drop('real',axis=1)

In [12]:
Y=df['real']

In [13]:
#Data Preprocessing
vocab_size = 10000
messages = X.copy()
messages.reset_index(inplace=True)


In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [60]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review=review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review =' '.join(review)
  corpus.append(review)

In [61]:
corpus[:30]

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang daughter kati complet leav son b

WITH MACHINE LEARNING ALGORITHM /WITHOUT **LSTM**

In [62]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [77]:
  # CountVectorizer/BOW
  cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
  tfid = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
  X_CV=cv.fit_transform(corpus).toarray()
  X_tfid = tfid.fit_transform(corpus).toarray()

In [78]:
X_CV[0]

array([0, 0, 0, ..., 0, 0, 0])

In [79]:
X_tfid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [80]:
X_CV.shape#5000 are feature/column names

(22866, 5000)

In [81]:
X_tfid.shape

(22866, 5000)

In [82]:
cv.get_feature_names_out()[30:90]# we took ngram_range(1,3). therefore it takes 1 word or 2 words or3 words

array(['acm award', 'acn', 'across', 'act', 'action', 'actor', 'actress',
       'actual', 'ad', 'adam', 'adam levin', 'adam rippon', 'add',
       'addict', 'address', 'adel', 'admit', 'adopt', 'ador',
       'ador photo', 'adult', 'adventur', 'advic', 'af', 'affair',
       'affleck', 'affleck jennif', 'affleck jennif garner',
       'affleck lindsay', 'affleck lindsay shooku', 'afford', 'afraid',
       'agdal', 'age', 'agent', 'ago', 'agostino', 'agre', 'agreement',
       'aguilera', 'ahead', 'aid', 'air', 'airlin', 'airport', 'aisl',
       'al', 'alba', 'album', 'alcohol', 'aldean', 'alec', 'alec baldwin',
       'alert', 'alex', 'alex rodriguez', 'alexa', 'alexand', 'alexandra',
       'alexi'], dtype=object)

In [84]:
tfid.get_feature_names_out()[30:90]

array(['acm award', 'acn', 'across', 'act', 'action', 'actor', 'actress',
       'actual', 'ad', 'adam', 'adam levin', 'adam rippon', 'add',
       'addict', 'address', 'adel', 'admit', 'adopt', 'ador',
       'ador photo', 'adult', 'adventur', 'advic', 'af', 'affair',
       'affleck', 'affleck jennif', 'affleck jennif garner',
       'affleck lindsay', 'affleck lindsay shooku', 'afford', 'afraid',
       'agdal', 'age', 'agent', 'ago', 'agostino', 'agre', 'agreement',
       'aguilera', 'ahead', 'aid', 'air', 'airlin', 'airport', 'aisl',
       'al', 'alba', 'album', 'alcohol', 'aldean', 'alec', 'alec baldwin',
       'alert', 'alex', 'alex rodriguez', 'alexa', 'alexand', 'alexandra',
       'alexi'], dtype=object)

In [88]:
from sklearn.model_selection import train_test_split
X_train_CV, X_test_CV,Y_train_CV,Y_test_CV = train_test_split(X_CV,Y,test_size=0.33, random_state=42)
X_train_tfid, X_test_tfid,Y_train_tfid,Y_test_tfid = train_test_split(X_tfid,Y,test_size=0.33, random_state=42)

In [89]:
X_CV_df = pd.DataFrame(X_train_CV, columns=cv.get_feature_names_out())
X_CV_df.head()

,aaron,aaron carter,aaron rodger,ab,abbey,abbi,abbi lee,abbi lee miller,abc,abc news,...,zayn malik,zendaya,zeta,zeta jone,zig,zoe,zoe saldana,zolciak,zolciak biermann,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
X_tfid_df = pd.DataFrame(X_train_tfid,columns=tfid.get_feature_names_out())
X_tfid_df.head()

,aaron,aaron carter,aaron rodger,ab,abbey,abbi,abbi lee,abbi lee miller,abc,abc news,...,zayn malik,zendaya,zeta,zeta jone,zig,zoe,zoe saldana,zolciak,zolciak biermann,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# Training the model using ML algorithm "Multinomail NB Classifier"
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [91]:
classifier.fit(X_train_CV,Y_train_CV)
Y_pred_CV = classifier.predict(X_test_CV)

In [93]:
classifier.fit(X_train_tfid,Y_train_tfid)
Y_pred_tfid = classifier.predict(X_test_tfid)

In [95]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("Accuracy of BOW= %0.3f" % accuracy_score(Y_test_CV,Y_pred_CV))
print("Confusion Matrix of BOW",confusion_matrix(Y_test_CV,Y_pred_CV))


Accuracy of BOW= 0.822
Confusion Matrix of BOW [[1253  589]
 [ 754 4950]]


In [96]:
print("Accuracy of TFID= %0.3f" % accuracy_score(Y_test_tfid,Y_pred_tfid))
print("Confusion Matrix of TFID",confusion_matrix(Y_test_tfid,Y_pred_tfid))

Accuracy of TFID= 0.838
Confusion Matrix of TFID [[ 889  953]
 [ 267 5437]]


Hyper Parameter Tunning for BOW


In [97]:
previous_score=0
import numpy as np

for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train_CV,Y_train_CV)
    Y_pred_CV=sub_classifier.predict(X_test_CV)
    score = accuracy_score(Y_test_CV, Y_pred_CV)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:629: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


Alpha: 0.0, Score : 0.8240127219719057
Alpha: 0.1, Score : 0.8246753246753247
Alpha: 0.2, Score : 0.8246753246753247
Alpha: 0.30000000000000004, Score : 0.8246753246753247
Alpha: 0.4, Score : 0.8242777630532733
Alpha: 0.5, Score : 0.8240127219719057
Alpha: 0.6000000000000001, Score : 0.8232175987278028
Alpha: 0.7000000000000001, Score : 0.8221574344023324
Alpha: 0.8, Score : 0.8218923933209648
Alpha: 0.9, Score : 0.8220249138616486


Hyper Parameter Tunning for TFID

In [98]:
previous_score=0
import numpy as np

for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train_tfid,Y_train_tfid)
    Y_pred_tfid=sub_classifier.predict(X_test_tfid)
    score = accuracy_score(Y_test_tfid, Y_pred_tfid)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:629: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


Alpha: 0.0, Score : 0.8405777895573814
Alpha: 0.1, Score : 0.8420355155049033
Alpha: 0.2, Score : 0.8429631592896899
Alpha: 0.30000000000000004, Score : 0.8438908030744765
Alpha: 0.4, Score : 0.8436257619931089
Alpha: 0.5, Score : 0.8436257619931089
Alpha: 0.6000000000000001, Score : 0.8423005565862709
Alpha: 0.7000000000000001, Score : 0.8413729128014842
Alpha: 0.8, Score : 0.8399151868539624
Alpha: 0.9, Score : 0.8383249403657567


With **LSTM**

In [99]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense

In [100]:
#one Hot Encoding
onehot_repres = [one_hot(words,vocab_size)for words in corpus]

In [101]:
# Padding
sent_length =20
pre_embedding = pad_sequences(onehot_repres,maxlen=sent_length,padding='pre')

In [102]:
pre_embedding[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2796,
       7955, 5555, 2028, 6249,  735, 4900, 7557, 5708, 1803], dtype=int32)

In [103]:
dim=40
from tensorflow.keras.layers import Dropout
model=Sequential()
model.add(Embedding(vocab_size,dim,input_length=sent_length))
# till now we done the word embedding. npw moving to steps of LSTM
model.add(Dropout(0.33))
model.add(LSTM(100))
model.add(Dropout(0.33))
model.add(Dense(1,activation='sigmoid'))# since it is a classification problem
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            400000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456501 (1.74 MB)
Trainable params: 456501 (1.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [104]:
import numpy as np
X_final = np.array(pre_embedding)
Y_final = np.array(Y)
X_final.shape,Y_final.shape

((22866, 20), (22866,))

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test = train_test_split(X_final,Y_final,test_size=0.33,random_state=42)

In [106]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
240/240 [==============================] - 19s 68ms/step - loss: 0.4694 - accuracy: 0.7946 - val_loss: 0.3943 - val_accuracy: 0.8310
Epoch 2/10
240/240 [==============================] - 19s 78ms/step - loss: 0.3442 - accuracy: 0.8510 - val_loss: 0.3942 - val_accuracy: 0.8353
Epoch 3/10
240/240 [==============================] - 16s 66ms/step - loss: 0.2994 - accuracy: 0.8708 - val_loss: 0.4063 - val_accuracy: 0.8222
Epoch 4/10
240/240 [==============================] - 17s 70ms/step - loss: 0.2706 - accuracy: 0.8852 - val_loss: 0.4289 - val_accuracy: 0.8341
Epoch 5/10
240/240 [==============================] - 13s 55ms/step - loss: 0.2416 - accuracy: 0.8995 - val_loss: 0.4565 - val_accuracy: 0.8281
Epoch 6/10
240/240 [==============================] - 13s 52ms/step - loss: 0.2210 - accuracy: 0.9072 - val_loss: 0.4877 - val_accuracy: 0.8280
Epoch 7/10
240/240 [==============================] - 15s 64ms/step - loss: 0.1984 - accuracy: 0.9181 - val_loss: 0.4959 - val_accuracy:

In [107]:
Y_pred = model.predict(X_test)
Y_pred=np.round(Y_pred).astype(int)


236/236 [==============================] - 3s 9ms/step


In [108]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))

[[1074  768]
 [ 716 4988]]
0.8033395176252319
